In [1]:
import pandas as pd
import numpy as np
from collections import Counter
import re

import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.ticker import MaxNLocator

%matplotlib inline

In [2]:
# reference:
ref_annot = pd.read_csv('../data/raw/ref_annot.csv')

In [3]:
REGION_SIZE = 50

In [4]:
WINDOW_SIZE = 10

In [5]:
g4conf = pd.read_csv('../data/processed/g4conf.csv')

In [11]:
g4conf['G4'].value_counts()

G4
G4     43
UG4    31
Name: count, dtype: int64

# Data preparation 
## Step 1
- auto-annotate G4 by G4Hunter (https://bioinformatics.cruk.cam.ac.uk/G4Hunter/)  
- *verificate auto-annotate G4 in human RefSeq using data from africa paper and exel*  

Dataset from paper [DOI:10.1093/nar/gkw006](DOI:10.1093/nar/gkw006) (G4Hunter and *in vitro* checking G4)

In [5]:
datasetG4 = pd.read_excel("../data/raw/SupplementaryTableS2.xlsx")

C:\Users\777\anaconda3\lib\site-packages\openpyxl\worksheet\_read_only.py:79: UserWarning: Conditional Formatting extension is not supported and will be removed
  for idx, row in parser.parse():
C:\Users\777\anaconda3\lib\site-packages\openpyxl\worksheet\_read_only.py:79: UserWarning: Unknown extension is not supported and will be removed
  for idx, row in parser.parse():


In [6]:
africaG4 = datasetG4[datasetG4['Type'] != 'notG4'][['seqnames','start','end', 'width', 'strand', 'seq', 'G4Hscore', 'Type']]

In [7]:
# africaG4.to_excel('data/processed/africaG4.xlsx')

In [8]:
refG4 = pd.read_table('../data/raw/NC_012920.1 Homo sapiens mitochondrion, complete genome_DNA_hl=1_k=25_G4Hseeked_2023-09-22.txt')

In [9]:
# refG4.to_excel('data/processed/refG4.xlsx')

## Manually fill in the data  
Compare two datasets africaG4, refG4  
If G4 present in both datasets put **1** in column *confirm*

## Step 2
### Interval size analysis  

In [10]:
refG4 = pd.read_excel('../data/processed/refG4.xlsx')

In [11]:
g4conf = refG4[refG4['confirmed'] == 1]

In [12]:
bp_untill_next_G4 = g4conf['start'].drop(1).values - g4conf['end'].drop(93).values
bp_untill_next_G4 = bp_untill_next_G4.tolist()
bp_untill_next_G4.append(294)  #294 bp between last G4 and first G4

In [13]:
bp_untill_previous_G4 = g4conf['start'].drop(1).values - g4conf['end'].drop(93).values
bp_untill_previous_G4 = bp_untill_previous_G4.tolist()
bp_untill_previous_G4.insert(0, 294)

In [14]:
g4conf['bp_untill_next_G4'] = bp_untill_next_G4
g4conf['bp_untill_previous_G4'] = bp_untill_previous_G4

C:\Users\777\AppData\Local\Temp\ipykernel_11760\2987571035.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  g4conf['bp_untill_next_G4'] = bp_untill_next_G4
C:\Users\777\AppData\Local\Temp\ipykernel_11760\2987571035.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  g4conf['bp_untill_previous_G4'] = bp_untill_previous_G4


Adding G4_id for each confirmed G4:

In [15]:
g4conf['G4_id'] = [*range(1,g4conf.shape[0]+1)]

C:\Users\777\AppData\Local\Temp\ipykernel_11760\169449425.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  g4conf['G4_id'] = [*range(1,g4conf.shape[0]+1)]


In [16]:
g4conf['up_win'] = pd.concat([g4conf[g4conf['bp_untill_previous_G4'] <= 100][['bp_untill_previous_G4', 'start']].diff(axis=1)['start'],\
                                 g4conf[g4conf['bp_untill_previous_G4'] > 100]['start']-100])

C:\Users\777\AppData\Local\Temp\ipykernel_11760\2099197523.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  g4conf['up_win'] = pd.concat([g4conf[g4conf['bp_untill_previous_G4'] <= 100][['bp_untill_previous_G4', 'start']].diff(axis=1)['start'],\


In [17]:
g4conf['down_win'] = pd.concat([g4conf[g4conf['bp_untill_next_G4'] <= 100][['end', 'bp_untill_next_G4']].sum(axis=1), \
                               g4conf[g4conf['bp_untill_next_G4'] > 100]['end']+100])

C:\Users\777\AppData\Local\Temp\ipykernel_11760\461284617.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  g4conf['down_win'] = pd.concat([g4conf[g4conf['bp_untill_next_G4'] <= 100][['end', 'bp_untill_next_G4']].sum(axis=1), \


In [18]:
# g4conf.to_csv('../data/processed/g4conf.csv', index=False)

## Step 2  
### New interval size:
- Take all confirmed G4   
- **Dont sort G4 by size between G4 >= 50 bp**  
- Obtain different interval size (0-100bp) between G4  

In [19]:
# G4_id = 38 hasnt downstream windows
# G4_id = 39 hasnt upsteram windows

In [20]:
g4conf[(g4conf['G4_id'] == 38) | (g4conf['G4_id'] == 39)]

,seqnames,start,end,width,strand,score,max_score,threshold,window,sequence,confirmed,G4,note,start-100bp,end+100bp,bp_untill_next_G4,bp_untill_previous_G4,G4_id,up_win,down_win
52,"NC_012920.1 Homo sapiens mitochondrion, comple...",9243,9262,20,-,-1.65,-1.32,1,25,CCCAGCCCATGACCCCTAAC,1,UG4,less than africa,9143,9362,6,285,38,9143,9268
53,"NC_012920.1 Homo sapiens mitochondrion, comple...",9268,9293,26,-,-1.15,-1.12,1,25,CCCTCTCAGCCCTCCTAATGACCTCC,1,UG4,NaN,9168,9393,212,6,39,9262,9393


In [21]:
data_up = []

for up_win, start, G4_id in zip(g4conf['up_win'], g4conf['start'], g4conf['G4_id']):
    
    count_window = start - up_win - WINDOW_SIZE + 1  # count window in upstream interval (before G4)
    number_of_win = count_window + 1                 # reverse counting of windows; the closer to G4, the smaller the number
    
    for i in range(up_win, start):
        row = []
        
        if i+WINDOW_SIZE <= start:
            window = [*range(i, i + WINDOW_SIZE)]
        else:
            break
#         print(i, window)    
        window_seq = ''
        for pos in window:
            if pos != 3107:
                window_seq += str(ref_annot[ref_annot['Pos'] == pos].Nuc.values[0])
            else:
                window_seq += ''
#         print(pos, window_seq)

#       ref_annot[ref_annot['Pos'].isin(window)].Nuc.value_counts()
        
        number_of_win -= 1
        
        row.append(G4_id)
        row.append('up')
        row.append(number_of_win)
        row.append(window[0])
        row.append(window[-1])
        row.append(window_seq)
        
        data_up.append(row)
# print(data_up)

In [22]:
data_down = []

for end, down_win, G4_id in zip(g4conf['end'], g4conf['down_win'], g4conf['G4_id']):
    
    number_of_win = 0
    
    for i in range(end, down_win):
        row = []
        
        if i >= g4conf['end'].iloc[-1] | i <= ref_annot['Pos'].iloc[-1]:
            window = [*range(i, ref_annot['Pos'].iloc[-1]+1)]
            remain = WINDOW_SIZE - (ref_annot['Pos'].iloc[-1] - i)
            window += [*range(ref_annot['Pos'].iloc[0], remain)]
        
        elif i > ref_annot['Pos'].iloc[-1] and i <= down_win - WINDOW_SIZE :
            i = i - ref_annot['Pos'].iloc[-1]
            window = [*range(i, i + WINDOW_SIZE)]
        
        elif i+WINDOW_SIZE < down_win:
            window = [*range(i, i + WINDOW_SIZE)]
               
        else:
            break
            
        window_seq = ''
        for pos in window:
            window_seq += ref_annot[ref_annot['Pos'] == pos].Nuc.values[0]
#         print(window[0], window_seq)

#     ref_annot[ref_annot['Pos'].isin(window)].Nuc.value_counts()

        number_of_win += 1

        row.append(G4_id)
        row.append('down')
        row.append(number_of_win)
        row.append(window[0])
        row.append(window[-1])
        row.append(window_seq)
        
        data_down.append(row)
# print(data_down)

In [23]:
data = data_up + data_down

In [24]:
df_win = pd.DataFrame(data=data, columns = ['G4_id', 'side', 'number_of_win', 'window_start', 'window_end', 'sequence'])

In [25]:
nuc_count_win = df_win['sequence'].apply(lambda x: Counter(x))

In [26]:
nuc_count_df_win = pd.DataFrame.from_records(nuc_count_win).sort_index(axis=1)

In [27]:
df_win = df_win.join(nuc_count_df_win).fillna(0)

### Finished dataset with nucleotide frequencies in ref_seq windows 

In [28]:
# df_win.to_csv('../data/processed/df_win.csv', index=False)

# Data preparation around G4 for Cancer 
### Observed mutations Cancer dataset:

df_win_cancer - observed mutations from Cancer dataset with nucleotide frequencies from df_win

Add mutations from Cancer dataset:

In [29]:
df_cancers = pd.read_csv('../data/raw/mutations.csv')

In [30]:
Mut_win = []
for window_start, window_end in zip(df_win['window_start'], df_win['window_end']):
    row = [*range(window_start, window_end)]
    Mut_win.append(str(df_cancers[df_cancers['Pos'].isin(row)].Mut.values))

In [31]:
Mut_win_df = pd.DataFrame(Mut_win)

In [32]:
Mut_win_df.head()

,0
0,['A[A>G]C' 'A[A>G]C' 'A[A>G]C' 'A[A>G]C' 'A[A>...
1,['A[A>G]C' 'A[A>G]C' 'A[A>G]C' 'A[A>G]C' 'A[A>...
2,['A[A>G]C' 'A[A>G]C' 'A[A>G]C' 'A[A>G]C' 'A[A>...
3,['A[A>G]C' 'A[A>G]C' 'A[A>G]C' 'A[A>G]C' 'A[A>...
4,['A[A>G]C' 'A[A>G]C' 'A[A>G]C' 'A[A>G]C' 'A[A>...


In [33]:
Mut_win_df[0] = Mut_win_df[0].str.findall("[A-T]>[A-T]")

In [34]:
obsMutwin = pd.DataFrame.from_records(Mut_win_df[0].apply(lambda x: Counter(x))).fillna(0).sort_index(axis=1)

In [35]:
df_win_cancer = df_win.join(obsMutwin)

In [36]:
df_win_cancer['number_of_Mut'] = df_win_cancer[['A>C','A>G','A>T',
                                                'C>A','C>G','C>T',
                                                'G>A','G>C','G>T',
                                                'T>A','T>C','T>G']].sum(axis=1)

### Finished dataset with observed mutations from cancer_df:

In [37]:
# df_win_cancer.to_csv('../data/processed/df_win_cancer.csv', index=False)

# Data preparation around G4 for Human Global Tree

In [38]:
df_hgt = pd.read_table('../data/raw/fulltreeCodons.csv', sep=';')

In [40]:
with open('../data/raw/hgt_alignment.blast') as fin:
    while True:
        line = fin.readline()
        if 'Strand=Plus/Plus' in line:
            fin.readline()
            break
    qseq = ''
    rods = ''
    sseq = ''
    query, sbjct = 'Query', 'Sbjct'
    i = 0
    size = 60
    start = 14
    while query.startswith('Query') and sbjct.startswith('Sbjct'):
        i += 1
        query = fin.readline().strip()
        cur_rods = fin.readline()
        sbjct = fin.readline().strip()

        if len(query[start:start+size]) == size:
            qseq += query[start:start+size]
            sseq += sbjct[start:start+size]
            rods += cur_rods[start:start+size]
        else:
            pos = re.search('Query\s{2}(\d+)\s+[\w-]+\s+(\d+)', query).groups()
            size = int(pos[1]) - int(pos[0]) + 1
            
            qseq += query[start:start+size]
            sseq += sbjct[start:start+size]
            rods += cur_rods[start:start+size]
            break
            
        fin.readline()

In [41]:
print('sseq =', len(sseq), 'qseq =', len(qseq))

sseq = 16568 qseq = 16568


In [42]:
sseq = 'G' + sseq
qseq = 'G' + qseq

In [43]:
print('sseq =', len(sseq), 'qseq =', len(qseq))

sseq = 16569 qseq = 16569


In [44]:
dict_hgt = {i: qseq[i] for i in range(len(qseq))}
dict_ref = {i: sseq[i] for i in range(len(sseq))}

In [45]:
df_ali_hgt_to_ref = pd.DataFrame.from_dict(dict_ref, orient='index', columns=['nuc_ref']) \
    .reset_index() \
    .rename(columns={'index':'Pos'})

In [46]:
df_ali_hgt_to_ref['nuc_hgt'] = df_ali_hgt_to_ref['Pos'].map(dict_hgt)

In [47]:
df_ali_hgt_to_ref['Pos'] = range(1, len(df_ali_hgt_to_ref['Pos'])+1)

In [48]:
df_ali_hgt_to_ref = df_ali_hgt_to_ref.replace({'c':'C', 'n':'N', 't':'T', 'g':'G', 'a':'A'})

In [ ]:
df_unique = df_hgt[['ref_pos', 'nuc_ref_in_ali']].sort_values(by='ref_pos').drop_duplicates(subset=['ref_pos'])

In [50]:
df_unique = df_unique.reset_index(drop=True)

In [51]:
df_unique_sort = df_unique[~((df_unique['nuc_ref_in_ali'] == '-') | (df_unique['nuc_ref_in_ali'] == 'N'))][['ref_pos', 'nuc_ref_in_ali']]

In [52]:
df_unique_sort = df_unique_sort.reset_index(drop=True)

In [54]:
# Detele all posirions with N, -, and first position
a = df_ali_hgt_to_ref[~((df_ali_hgt_to_ref['nuc_hgt'] == 'N') | 
                        (df_ali_hgt_to_ref['nuc_hgt'] == '-') | 
                        (df_ali_hgt_to_ref['Pos'] == 1))].reset_index()

In [55]:
df_pos_ref_to_hgt = pd.concat([a, df_unique_sort], axis=1).rename(columns={'ref_pos':'hgt_ref_pos'}).set_index('index')

In [56]:
df_pos_ref_to_hgt.index.name = None

In [57]:
df_ali_hgt_to_ref = df_ali_hgt_to_ref.join(df_pos_ref_to_hgt, how='left', rsuffix='2') \
    .drop(columns=['Pos2', 'nuc_ref2', 'nuc_hgt2', 'nuc_hgt']) \
    .astype({'hgt_ref_pos':'Int64'})

In [58]:
# Add N in table
df_ali_hgt_to_ref.iloc[3106, df_ali_hgt_to_ref.columns.get_loc('hgt_ref_pos')] = 3106
df_ali_hgt_to_ref.iloc[3106, df_ali_hgt_to_ref.columns.get_loc('nuc_ref_in_ali')] = 'N'

In [59]:
for start, end, G4_id in zip(g4conf['start'], g4conf['end'], g4conf['G4_id']):
    df_ali_hgt_to_ref.loc[df_ali_hgt_to_ref.Pos.isin([*range(start, end+1)]),'G4'] = 1 #is position G4? 1-yes, 0-no
    df_ali_hgt_to_ref.loc[df_ali_hgt_to_ref.Pos.isin([*range(start, end+1)]),'G4_id'] = G4_id

In [60]:
for up_win, start, G4_id in zip(g4conf['up_win'], g4conf['start'], g4conf['G4_id']):
    df_ali_hgt_to_ref.loc[df_ali_hgt_to_ref.Pos.isin([*range(up_win, start+1)]),'up_win'] = G4_id

In [61]:
for end, down_win, G4_id in zip(g4conf['end'], g4conf['down_win'], g4conf['G4_id']):
    df_ali_hgt_to_ref.loc[df_ali_hgt_to_ref.Pos.isin([*range(end, down_win+1)]),'down_win'] = G4_id

In [62]:
df_ali_hgt_to_ref[['G4', 'G4_id', 'up_win', 'down_win']] = df_ali_hgt_to_ref[['G4',  'G4_id', 
                                                                              'up_win', 'down_win']].fillna(0)

In [63]:
df_ali_hgt_to_ref[['hgt_ref_pos', 'Pos', 'G4_id', 'G4', 'up_win', 'down_win']] \
    .rename(columns={'hgt_ref_pos':'ref_pos'}).head()

,ref_pos,Pos,G4_id,G4,up_win,down_win
0,<NA>,1,0.0,0.0,0.0,0.0
1,1,2,0.0,0.0,0.0,0.0
2,2,3,0.0,0.0,0.0,0.0
3,3,4,0.0,0.0,0.0,0.0
4,4,5,0.0,0.0,0.0,0.0


In [64]:
df_hgt_g4 = df_hgt \
    .merge(df_ali_hgt_to_ref[['hgt_ref_pos', 'Pos', 'G4_id', 'G4', 'up_win', 'down_win']] \
           .rename(columns={'hgt_ref_pos':'ref_pos'}), how='left') \
    .sort_values(by='ref_pos')

In [65]:
df_hgt_g4['Mut'] = df_hgt_g4.ancestor.str[2] + '>' + df_hgt_g4.descendant.str[2]

In [66]:
# df_hgt_g4.to_csv('data/processed/df_hgt_g4.csv', index=False)

### Collect observed mutations from HGT

In [67]:
df_hgt_g4 = pd.read_csv('data/processed/df_hgt_g4.csv')

C:\Users\777\AppData\Local\Temp\ipykernel_13856\3731285902.py:1: DtypeWarning: Columns (10,11,12,14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  df_hgt_g4 = pd.read_csv('data/processed/df_hgt_g4.csv')


In [68]:
# Extract mutations without gaps (-) and other (R, M, N, ect) bases in mut position:
df_hgt_g4 = df_hgt_g4[
    (df_hgt_g4.ancestor.str.lower().str.fullmatch("[acgt]{5}")) &
    (df_hgt_g4.descendant.str.lower().str.fullmatch("[acgt]{5}"))]

In [69]:
obs_mut_hgt_up = []
for up_win, Pos, Mut in zip(df_hgt_g4['up_win'], df_hgt_g4['Pos'], df_hgt_g4['Mut']):
    row = []
    
    if up_win > 0:
        row.append(up_win)
        row.append('up')
        row.append(Pos)
        row.append(Mut)
    else:
        continue
    
    obs_mut_hgt_up.append(row)

In [70]:
obs_mut_hgt_down = []
for down_win, Pos, Mut in zip(df_hgt_g4['down_win'], df_hgt_g4['Pos'], df_hgt_g4['Mut']):
    row = []
    
    if down_win > 0:
        row.append(down_win)
        row.append('down')
        row.append(Pos)
        row.append(Mut)
    else:
        continue
    
    obs_mut_hgt_down.append(row)

In [71]:
obs_mut_hgt = obs_mut_hgt_up + obs_mut_hgt_down

In [72]:
df_obs_mut_hgt = pd.DataFrame(data=obs_mut_hgt, columns=['G4_id', 'side', 'Pos', 'Mut'])

In [73]:
df_win = pd.read_csv('data/processed/df_win.csv')

In [74]:
Mut_hgt_win = []
for window_start, window_end in zip(df_win['window_start'], df_win['window_end']):
    row = []
    
    mut = df_obs_mut_hgt[(df_obs_mut_hgt['Pos'] >= window_start) & (df_obs_mut_hgt['Pos'] <= window_end)].Mut.tolist()
    
    row.append(mut)
    
    Mut_hgt_win.append(row)

In [75]:
df_Mut_hgt_win = pd.DataFrame(data=Mut_hgt_win)

In [77]:
obsMutG4_hgt = pd.DataFrame.from_records(df_Mut_hgt_win[0].apply(lambda x: Counter(x))).fillna(0).sort_index(axis=1)

In [78]:
df_win_hgt = df_win.join(obsMutG4_hgt)

In [79]:
df_win_hgt['number_of_Mut'] = df_win_hgt[['A>C','A>G','A>T','C>A','C>G','C>T','G>A','G>C','G>T','T>A','T>C','T>G']].sum(axis=1)

In [80]:
# df_win_hgt.to_csv('data/processed/df_win_hgt.csv', index=False)

# Data preparation within G4

In [6]:
g4conf = pd.read_csv('../data/processed/g4conf.csv')

In [82]:
data_within = []

for start, end, G4_id, sequence in zip(g4conf['start'], g4conf['end'], g4conf['G4_id'], g4conf['sequence']):
    
    number_of_win = 0
    
    for i, j in zip(range(start, end), range(len(sequence))):
        row = []
        
        if i+WINDOW_SIZE < end:
            window = [*range(i, i + WINDOW_SIZE)]       
        else:
            break
            
        window_seq = ''
        window_seq += sequence[j: j + WINDOW_SIZE]

        
        number_of_win += 1

        row.append(G4_id)
        row.append('within')
        row.append(number_of_win)
        row.append(window[0])
        row.append(window[-1])
        row.append(window_seq)
        
        data_within.append(row)
# print(data_down)

In [83]:
df_win_within = pd.DataFrame(data=data_within, 
                             columns = ['G4_id', 'side', 'number_of_win', 'window_start', 'window_end', 'sequence'])

In [84]:
nuc_count_win_within = df_win_within['sequence'].apply(lambda x: Counter(x))

In [85]:
nuc_count_df_win_within = pd.DataFrame.from_records(nuc_count_win_within).sort_index(axis=1)

In [86]:
df_win_within = df_win_within.join(nuc_count_df_win_within).fillna(0)

In [8]:
# df_win_within.to_csv('../data/processed/df_win_within.csv', index=False)

# Data preparation Cancer within G4

In [88]:
df_cancers = pd.read_csv('data/raw/mutations.csv')
df_win_within = pd.read_csv('data/processed/df_win_within.csv')

In [89]:
Mut_win_within = []
for window_start, window_end in zip(df_win_within['window_start'], df_win_within['window_end']):
    row = [*range(window_start, window_end)]
    Mut_win_within.append(str(df_cancers[df_cancers['Pos'].isin(row)].Mut.values))

In [90]:
Mut_win_df_within = pd.DataFrame(Mut_win_within)

In [91]:
Mut_win_df_within[0] = Mut_win_df_within[0].str.findall("[A-T]>[A-T]")

In [92]:
obsMutwin_within = pd.DataFrame.from_records(Mut_win_df_within[0].apply(lambda x: Counter(x))).fillna(0).sort_index(axis=1)

In [93]:
df_win_cancer_within = df_win_within.join(obsMutwin_within)

In [94]:
df_win_cancer_within['number_of_Mut'] = df_win_cancer_within[['A>C','A>G','A>T',
                                                              'C>A','C>G','C>T',
                                                              'G>A','G>C','G>T',
                                                              'T>A','T>C','T>G']].sum(axis=1)

In [95]:
df_win_cancer_within.head()

,G4_id,side,number_of_win,window_start,window_end,sequence,A,C,G,T,...,C>A,C>G,C>T,G>A,G>C,G>T,T>A,T>C,T>G,number_of_Mut
0,1,within,1,285,294,CAAAAAATTT,6.0,1.0,0.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,2.0
1,1,within,2,286,295,AAAAAATTTC,6.0,1.0,0.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,4.0
2,1,within,3,287,296,AAAAATTTCC,5.0,2.0,0.0,3.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,4.0,0.0,5.0
3,1,within,4,288,297,AAAATTTCCA,5.0,2.0,0.0,3.0,...,0.0,0.0,4.0,0.0,0.0,0.0,0.0,4.0,0.0,8.0
4,1,within,5,289,298,AAATTTCCAC,4.0,3.0,0.0,3.0,...,0.0,0.0,4.0,0.0,0.0,0.0,0.0,4.0,0.0,10.0


In [96]:
# df_win_cancer_within.to_csv('data/processed/df_win_cancer_within.csv', index=False)

# Data preparation HGT within G4

In [97]:
df_hgt_g4 = pd.read_csv('data/processed/df_hgt_g4.csv')
df_win_within = pd.read_csv('data/processed/df_win_within.csv')

C:\Users\777\AppData\Local\Temp\ipykernel_13856\486489025.py:1: DtypeWarning: Columns (10,11,12,14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  df_hgt_g4 = pd.read_csv('data/processed/df_hgt_g4.csv')


In [98]:
# Extract mutations without gaps (-) and other (R, M, N, ect) bases in mut position:
df_hgt_g4 = df_hgt_g4[
    (df_hgt_g4.ancestor.str.lower().str.fullmatch("[acgt]{5}")) &
    (df_hgt_g4.descendant.str.lower().str.fullmatch("[acgt]{5}"))]

In [99]:
obs_mut_hgt_within = []
for G4, G4_id, Pos, Mut in zip(df_hgt_g4['G4'], df_hgt_g4['G4_id'], df_hgt_g4['Pos'], df_hgt_g4['Mut']):
    row = []
    
    if G4 > 0:
        row.append(G4_id)
        row.append('within')
        row.append(Pos)
        row.append(Mut)
    else:
        continue
    
    obs_mut_hgt_within.append(row)

In [100]:
df_obs_mut_hgt_within = pd.DataFrame(data=obs_mut_hgt_within, columns=['G4_id', 'side', 'Pos', 'Mut'])

In [101]:
Mut_hgt_win_within = []
for window_start, window_end in zip(df_win_within['window_start'], df_win_within['window_end']):
    row = []
    
    mut = df_obs_mut_hgt_within[(df_obs_mut_hgt_within['Pos'] >= window_start) & 
        (df_obs_mut_hgt_within['Pos'] <= window_end)]['Mut'].tolist()

    row.append(mut)
    
    Mut_hgt_win_within.append(row)

In [102]:
df_Mut_hgt_win_within = pd.DataFrame(data=Mut_hgt_win_within)

In [103]:
obsMutG4_hgt_within = pd.DataFrame.from_records(df_Mut_hgt_win_within[0].apply(lambda x: Counter(x))).fillna(0).sort_index(axis=1)

In [104]:
df_win_hgt_within = df_win_within.join(obsMutG4_hgt_within)

In [105]:
df_win_hgt_within['number_of_Mut'] = df_win_hgt_within[['A>C','A>G','A>T',
                                                        'C>A','C>G','C>T',
                                                        'G>A','G>C','G>T',
                                                        'T>A','T>C','T>G']].sum(axis=1)

In [106]:
df_win_hgt_within.head()

,G4_id,side,number_of_win,window_start,window_end,sequence,A,C,G,T,...,C>A,C>G,C>T,G>A,G>C,G>T,T>A,T>C,T>G,number_of_Mut
0,1,within,1,285,294,CAAAAAATTT,6.0,1.0,0.0,3.0,...,0.0,0.0,76.0,5.0,0.0,0.0,8.0,88.0,0.0,189.0
1,1,within,2,286,295,AAAAAATTTC,6.0,1.0,0.0,3.0,...,26.0,0.0,128.0,5.0,0.0,0.0,9.0,140.0,0.0,346.0
2,1,within,3,287,296,AAAAATTTCC,5.0,2.0,0.0,3.0,...,28.0,0.0,152.0,5.0,0.0,0.0,9.0,163.0,0.0,397.0
3,1,within,4,288,297,AAAATTTCCA,5.0,2.0,0.0,3.0,...,34.0,0.0,152.0,72.0,0.0,0.0,9.0,163.0,0.0,548.0
4,1,within,5,289,298,AAATTTCCAC,4.0,3.0,0.0,3.0,...,35.0,0.0,152.0,67.0,0.0,0.0,9.0,163.0,0.0,539.0


In [107]:
# df_win_hgt_within.to_csv('data/processed/df_win_hgt_within.csv', index=False)

# Data preparation within G4 with sides

In [6]:
g4conf = pd.read_csv('../data/processed/g4conf.csv')

In [7]:
data_within_start = []

for start, end, G4_id, sequence in zip(g4conf['start'], g4conf['end'], g4conf['G4_id'], g4conf['sequence']):
    
    number_of_win = 0
    
    for i, j in zip(range(start, end), range(len(sequence))):
        row = []
        
        if i+WINDOW_SIZE <= end:
            window = [*range(i, i + WINDOW_SIZE)]       
        else:
            break
        
        # print(i, window, WINDOW_SIZE, end)    
        window_seq = ''
        window_seq += sequence[j: j + WINDOW_SIZE]

        
        number_of_win += 1

        row.append(G4_id)
        row.append('within_start')
        row.append(number_of_win)
        row.append(window[0])
        row.append(window[-1])
        row.append(window_seq)
        
        data_within_start.append(row)
# print(data_down)

In [8]:
data_within_end = []

for start, end, G4_id, sequence in zip(g4conf['start'], g4conf['end'], g4conf['G4_id'], g4conf['sequence']):
    
    number_of_win = 0
    
    for i, j in zip(range(end, start, -1), range(len(sequence))):
        row = []

        
        if i+WINDOW_SIZE > end:
            continue  
        elif (i+WINDOW_SIZE <= end):
            window = [*range(i, i + WINDOW_SIZE)]       
        else:
            continue
        # print(window)
        
        window_seq = ''
        window_seq += sequence[::-1][j-10:j+WINDOW_SIZE-10][::-1]   #window_seq from the end. works with '-10' 

        # print(i, j)
        
        number_of_win += 1

        row.append(G4_id)
        row.append('within_end')
        row.append(number_of_win)
        row.append(window[0])
        row.append(window[-1])
        row.append(window_seq)
        
        data_within_end.append(row)
# print(data_down)

In [9]:
data_within_sides = data_within_start + data_within_end

In [10]:
df_win_within_sides = pd.DataFrame(data=data_within_sides, 
                             columns = ['G4_id', 'side', 'number_of_win', 'window_start', 'window_end', 'sequence'])

In [11]:
df_win_within_sides[(df_win_within_sides['side'] == 'within_end') & (df_win_within_sides['G4_id'] == 1)]

,G4_id,side,number_of_win,window_start,window_end,sequence
2816,1,within_end,1,322,331,GCCACAGCAC
2817,1,within_end,2,321,330,GGCCACAGCA
2818,1,within_end,3,320,329,TGGCCACAGC
2819,1,within_end,4,319,328,CTGGCCACAG
2820,1,within_end,5,318,327,TCTGGCCACA
2821,1,within_end,6,317,326,TTCTGGCCAC
2822,1,within_end,7,316,325,CTTCTGGCCA
2823,1,within_end,8,315,324,GCTTCTGGCC
2824,1,within_end,9,314,323,CGCTTCTGGC
2825,1,within_end,10,313,322,CCGCTTCTGG


In [12]:
df_win_within_sides[(df_win_within_sides['side'] == 'within_start') & (df_win_within_sides['G4_id'] == 1)]

,G4_id,side,number_of_win,window_start,window_end,sequence
0,1,within_start,1,285,294,CAAAAAATTT
1,1,within_start,2,286,295,AAAAAATTTC
2,1,within_start,3,287,296,AAAAATTTCC
3,1,within_start,4,288,297,AAAATTTCCA
4,1,within_start,5,289,298,AAATTTCCAC
5,1,within_start,6,290,299,AATTTCCACC
6,1,within_start,7,291,300,ATTTCCACCA
7,1,within_start,8,292,301,TTTCCACCAA
8,1,within_start,9,293,302,TTCCACCAAA
9,1,within_start,10,294,303,TCCACCAAAC


In [62]:
nuc_count_win_within_sides = df_win_within_sides['sequence'].apply(lambda x: Counter(x))

In [63]:
nuc_count_df_win_within_sides = pd.DataFrame.from_records(nuc_count_win_within_sides).sort_index(axis=1)

In [64]:
df_win_within_sides = df_win_within_sides.join(nuc_count_df_win_within_sides).fillna(0)

In [66]:
# df_win_within_sides.to_csv('../data/processed/df_win_within_sides.csv', index=False)

# Within sides Cancer

In [31]:
df_cancers = pd.read_csv('../data/raw/mutations.csv')
df_win_within_sides = pd.read_csv('../data/processed/df_win_within_sides.csv')

In [68]:
Mut_win_within_sides = []
for window_start, window_end in zip(df_win_within_sides['window_start'], df_win_within_sides['window_end']):
    row = [*range(window_start, window_end)]
    Mut_win_within_sides.append(str(df_cancers[df_cancers['Pos'].isin(row)].Mut.values))

In [69]:
Mut_win_df_within_sides = pd.DataFrame(Mut_win_within_sides)

In [70]:
Mut_win_df_within_sides[0] = Mut_win_df_within_sides[0].str.findall("[A-T]>[A-T]")

In [71]:
obsMutwin_within_sides = pd.DataFrame.from_records(Mut_win_df_within_sides[0].apply(lambda x: Counter(x))) \
    .fillna(0) \
    .sort_index(axis=1)

In [72]:
df_win_cancer_within_sides = df_win_within_sides.join(obsMutwin_within_sides)

In [73]:
df_win_cancer_within_sides['number_of_Mut'] = df_win_cancer_within_sides[['A>C','A>G','A>T',
                                                              'C>A','C>G','C>T',
                                                              'G>A','G>C','G>T',
                                                              'T>A','T>C','T>G']].sum(axis=1)

In [74]:
df_win_cancer_within_sides.head()

,G4_id,side,number_of_win,window_start,window_end,sequence,A,C,G,T,...,C>A,C>G,C>T,G>A,G>C,G>T,T>A,T>C,T>G,number_of_Mut
0,1,within_start,1,285,294,CAAAAAATTT,6.0,1.0,0.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,2.0
1,1,within_start,2,286,295,AAAAAATTTC,6.0,1.0,0.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,4.0
2,1,within_start,3,287,296,AAAAATTTCC,5.0,2.0,0.0,3.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,4.0,0.0,5.0
3,1,within_start,4,288,297,AAAATTTCCA,5.0,2.0,0.0,3.0,...,0.0,0.0,4.0,0.0,0.0,0.0,0.0,4.0,0.0,8.0
4,1,within_start,5,289,298,AAATTTCCAC,4.0,3.0,0.0,3.0,...,0.0,0.0,4.0,0.0,0.0,0.0,0.0,4.0,0.0,10.0


In [76]:
# df_win_cancer_within_sides.to_csv('../data/processed/df_win_cancer_within_sides.csv', index=False)

# Within sides HGT

In [78]:
df_hgt_g4 = pd.read_csv('../data/processed/df_hgt_g4.csv')
df_win_within_sides = pd.read_csv('../data/processed/df_win_within_sides.csv')

C:\Users\777\AppData\Local\Temp\ipykernel_13260\1406226394.py:1: DtypeWarning: Columns (10,11,12,14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  df_hgt_g4 = pd.read_csv('../data/processed/df_hgt_g4.csv')


In [79]:
# Extract mutations without gaps (-) and other (R, M, N, ect) bases in mut position:
df_hgt_g4 = df_hgt_g4[
    (df_hgt_g4.ancestor.str.lower().str.fullmatch("[acgt]{5}")) &
    (df_hgt_g4.descendant.str.lower().str.fullmatch("[acgt]{5}"))]

In [80]:
obs_mut_hgt_within_sides = []
for G4, G4_id, Pos, Mut in zip(df_hgt_g4['G4'], df_hgt_g4['G4_id'], df_hgt_g4['Pos'], df_hgt_g4['Mut']):
    row = []
    
    if G4 > 0:
        row.append(G4_id)
        row.append('within')
        row.append(Pos)
        row.append(Mut)
    else:
        continue
    
    obs_mut_hgt_within_sides.append(row)

In [81]:
df_obs_mut_hgt_within_sides = pd.DataFrame(data=obs_mut_hgt_within_sides, columns=['G4_id', 'side', 'Pos', 'Mut'])

In [82]:
Mut_hgt_win_within_sides = []
for window_start, window_end in zip(df_win_within_sides['window_start'], df_win_within_sides['window_end']):
    row = []
    
    mut = df_obs_mut_hgt_within_sides[(df_obs_mut_hgt_within_sides['Pos'] >= window_start) & 
        (df_obs_mut_hgt_within_sides['Pos'] <= window_end)]['Mut'].tolist()

    row.append(mut)
    
    Mut_hgt_win_within_sides.append(row)

In [83]:
df_Mut_hgt_win_within_sides = pd.DataFrame(data=Mut_hgt_win_within_sides)

In [84]:
obsMutG4_hgt_within_sides = pd.DataFrame.from_records(df_Mut_hgt_win_within_sides[0].apply(lambda x: Counter(x))) \
    .fillna(0) \
    .sort_index(axis=1)

In [85]:
df_win_hgt_within_sides = df_win_within_sides.join(obsMutG4_hgt_within_sides)

In [86]:
df_win_hgt_within_sides['number_of_Mut'] = df_win_hgt_within_sides[['A>C','A>G','A>T',
                                                        'C>A','C>G','C>T',
                                                        'G>A','G>C','G>T',
                                                        'T>A','T>C','T>G']].sum(axis=1)

In [87]:
df_win_hgt_within_sides.head()

,G4_id,side,number_of_win,window_start,window_end,sequence,A,C,G,T,...,C>A,C>G,C>T,G>A,G>C,G>T,T>A,T>C,T>G,number_of_Mut
0,1,within_start,1,285,294,CAAAAAATTT,6.0,1.0,0.0,3.0,...,0.0,0.0,76.0,5.0,0.0,0.0,8.0,88.0,0.0,189.0
1,1,within_start,2,286,295,AAAAAATTTC,6.0,1.0,0.0,3.0,...,26.0,0.0,128.0,5.0,0.0,0.0,9.0,140.0,0.0,346.0
2,1,within_start,3,287,296,AAAAATTTCC,5.0,2.0,0.0,3.0,...,28.0,0.0,152.0,5.0,0.0,0.0,9.0,163.0,0.0,397.0
3,1,within_start,4,288,297,AAAATTTCCA,5.0,2.0,0.0,3.0,...,34.0,0.0,152.0,72.0,0.0,0.0,9.0,163.0,0.0,548.0
4,1,within_start,5,289,298,AAATTTCCAC,4.0,3.0,0.0,3.0,...,35.0,0.0,152.0,67.0,0.0,0.0,9.0,163.0,0.0,539.0


In [89]:
# df_win_hgt_within_sides.to_csv('../data/processed/df_win_hgt_within_sides.csv', index=False)

# Location of g4

In [58]:
g4_location = []
for G4_id, start, end in zip(g4conf['G4_id'], g4conf['start'], g4conf['end']):
    location = []
    row = []
    for i in range(start, end+1):
        # row.append(G4_id)
        # row.append(i)
        location.append(ref_annot[ref_annot['Pos'] == i]['Type'].to_string(index=False))
        
    row.append(G4_id)
    row.append('/'.join(list(set(location))))
    g4_location.append(row)

In [59]:
g4_location

[[1, 'D-loop'],
 [2, 'D-loop'],
 [3, 'D-loop/tRNA'],
 [4, 'rRNA'],
 [5, 'rRNA'],
 [6, 'tRNA/rRNA'],
 [7, 'rRNA'],
 [8, 'rRNA'],
 [9, 'rRNA'],
 [10, 'rRNA'],
 [11, 'CDS'],
 [12, 'CDS'],
 [13, 'CDS'],
 [14, 'CDS'],
 [15, 'CDS'],
 [16, 'CDS'],
 [17, 'tRNA'],
 [18, 'tRNA/CDS'],
 [19, 'CDS'],
 [20, 'CDS'],
 [21, 'CDS'],
 [22, 'CDS'],
 [23, 'CDS'],
 [24, 'tRNA/NaN/CDS'],
 [25, 'CDS'],
 [26, 'CDS'],
 [27, 'CDS'],
 [28, 'CDS'],
 [29, 'CDS'],
 [30, 'CDS'],
 [31, 'CDS'],
 [32, 'NaN/CDS/tRNA'],
 [33, 'CDS'],
 [34, 'CDS'],
 [35, 'CDS'],
 [36, 'CDS'],
 [37, 'CDS'],
 [38, 'CDS'],
 [39, 'CDS'],
 [40, 'CDS'],
 [41, 'CDS'],
 [42, 'CDS'],
 [43, 'CDS'],
 [44, 'CDS'],
 [45, 'CDS'],
 [46, 'CDS'],
 [47, 'CDS'],
 [48, 'CDS'],
 [49, 'CDS'],
 [50, 'CDS'],
 [51, 'CDS'],
 [52, 'CDS'],
 [53, 'CDS'],
 [54, 'CDS'],
 [55, 'CDS'],
 [56, 'CDS'],
 [57, 'CDS'],
 [58, 'CDS'],
 [59, 'CDS'],
 [60, 'CDS'],
 [61, 'CDS'],
 [62, 'CDS'],
 [63, 'CDS'],
 [64, 'CDS'],
 [65, 'CDS'],
 [66, 'CDS'],
 [67, 'CDS'],
 [68, 'CDS'],
 [69, '

In [60]:
df_g4_location = pd.DataFrame(data=g4_location, columns=['G4_id', 'location'])

In [61]:
df_g4_location

,G4_id,location
0,1,D-loop
1,2,D-loop
2,3,D-loop/tRNA
3,4,rRNA
4,5,rRNA
...,...,...
69,70,CDS
70,71,D-loop
71,72,D-loop
72,73,D-loop


In [65]:
df_g4_location = df_g4_location.set_index('G4_id')['location'].str.split('/', expand=True) \
    .stack() \
    .reset_index(name='location') \
    .drop('level_1', axis=1)

In [66]:
df_g4_location

,G4_id,location
0,1,D-loop
1,2,D-loop
2,3,D-loop
3,3,tRNA
4,4,rRNA
...,...,...
76,70,CDS
77,71,D-loop
78,72,D-loop
79,73,D-loop


In [64]:
# df_g4_location.to_csv('../data/processed/df_g4_location.csv', index=False)

In [7]:
g4conf.columns

Index(['seqnames', 'start', 'end', 'width', 'strand', 'score', 'max_score',
       'threshold', 'window', 'sequence', 'confirmed', 'G4', 'note',
       'start-100bp', 'end+100bp', 'start-50bp', 'end+50bp',
       'bp_untill_next_G4', 'bp_untill_previous_G4', 'G4_id', 'up_win',
       'down_win'],
      dtype='object')

In [13]:
df_g4_type = g4conf[['G4_id', 'G4']].rename(columns={'G4':'Type_of_G4'})

In [17]:
# df_g4_type.to_csv('../data/processed/df_g4_type.csv', index=False)

# G4 location *in vivo* (doi.org/10.1093/nar/gkad535)

In [34]:
G4_peaks = [[1, 2425, 3986], [2, 4330, 4624], [3, 5570, 7168], [4, 7139, 7559], [5, 7800, 8276], [6, 8958, 9869], [7, 12762, 13771]]

In [41]:
df_g4_vivo_peaks = pd.DataFrame(G4_peaks, columns=['number', 'start', 'end'])

In [42]:
df_g4_vivo_peaks

,number,start,end
0,1,2425,3986
1,2,4330,4624
2,3,5570,7168
3,4,7139,7559
4,5,7800,8276
5,6,8958,9869
6,7,12762,13771


In [57]:
# g4conf

In [52]:
g4_vivo = []
for start, end, G4_id in zip(g4conf['start'], g4conf['end'], g4conf['G4_id']):
    for p_start, p_end, number in zip(df_g4_vivo_peaks['start'], df_g4_vivo_peaks['end'], df_g4_vivo_peaks['number']):
        row = []
        if start in range(p_start, p_end) or end in range(p_start, p_end):
            row.append(number)
            row.append(G4_id)
            row.append(1)
            g4_vivo.append(row)

In [53]:
df_g4_vivo = pd.DataFrame(g4_vivo, columns=['peak_number', 'G4_id', 'in_vivo'])

In [54]:
df_g4_vivo

,peak_number,G4_id,in_vivo
0,1,9,1
1,1,10,1
2,1,11,1
3,1,12,1
4,1,13,1
5,2,17,1
6,2,18,1
7,3,24,1
8,3,25,1
9,3,26,1


In [56]:
# df_g4_vivo.to_csv('../data/processed/df_g4_vivo.csv', index=False)

# Check decrease of С>T  
Make new windows which cover all length G4 and +-25 bp 


In [7]:
g4conf

,seqnames,start,end,width,strand,score,max_score,threshold,window,sequence,...,note,start-100bp,end+100bp,start-50bp,end+50bp,bp_untill_next_G4,bp_untill_previous_G4,G4_id,up_win,down_win
0,"NC_012920.1 Homo sapiens mitochondrion, comple...",285,332,48,-,-1.250,-2.24,1,25,CAAAAAATTTCCACCAAACCCCCCCTCCCCCGCTTCTGGCCACAGCAC,...,more than africa,185,432,235,382,16,294,1,185,348
1,"NC_012920.1 Homo sapiens mitochondrion, comple...",348,378,31,-,-1.100,-1.16,1,25,CCAAACCCCAAAAACAAAGAACCCTAACACC,...,less than africa,248,478,298,428,40,16,2,332,418
2,"NC_012920.1 Homo sapiens mitochondrion, comple...",418,592,175,-,-1.150,-1.88,1,25,CACTTTTAACAGTCACCCCCCAACTAACACATTATTTTCCCCTCCC...,...,5 overlapping g4 in africa,318,692,368,642,193,40,3,378,692
3,"NC_012920.1 Homo sapiens mitochondrion, comple...",785,808,24,-,-1.210,-1.16,1,25,CGCTTAGCCTAGCCACACCCCCAC,...,NaN,685,908,735,858,146,193,4,685,908
4,"NC_012920.1 Homo sapiens mitochondrion, comple...",954,984,31,-,-1.390,-1.48,1,25,CACCCCCTCCCCAATAAAGCTAAAACTCACC,...,NaN,854,1084,904,1034,683,146,5,854,1084
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69,"NC_012920.1 Homo sapiens mitochondrion, comple...",15636,15683,48,-,-0.896,-1.16,1,25,CCATCCTCATCCTAGCAATAATCCCCATCCTCCATATATCCAAACAAC,...,more than africa,15536,15783,15586,15733,484,80,70,15556,15783
70,"NC_012920.1 Homo sapiens mitochondrion, comple...",16167,16211,45,-,-1.160,-1.80,1,25,CCCAATCCACATCAAAACCCCCTCCCCATGCTTACAAGCAAGTAC,...,more than africa,16067,16311,16117,16261,28,484,71,16067,16239
71,"NC_012920.1 Homo sapiens mitochondrion, comple...",16239,16301,63,-,-0.968,-1.40,1,25,CAACTGCAACTCCAAAGCCACCCCTCACCCACTAGGATACCAACAA...,...,more than africa,16139,16401,16189,16351,43,28,72,16211,16344
72,"NC_012920.1 Homo sapiens mitochondrion, comple...",16344,16382,39,-,-1.210,-1.48,1,25,CAGTCAAATCCCTTCTCGTCCCCATGGATGACCCCCCTC,...,more than africa,16244,16482,16294,16432,145,43,73,16301,16482


In [6]:
df_25bp = g4conf[(g4conf['bp_untill_previous_G4'] >= 25) & 
            (g4conf['bp_untill_next_G4'] >= 25) & 
            (g4conf['end'] + 25 <= 16569)]

In [7]:
df_25bp

,seqnames,start,end,width,strand,score,max_score,threshold,window,sequence,...,note,start-100bp,end+100bp,start-50bp,end+50bp,bp_untill_next_G4,bp_untill_previous_G4,G4_id,up_win,down_win
2,"NC_012920.1 Homo sapiens mitochondrion, comple...",418,592,175,-,-1.150,-1.88,1,25,CACTTTTAACAGTCACCCCCCAACTAACACATTATTTTCCCCTCCC...,...,5 overlapping g4 in africa,318,692,368,642,193,40,3,378,692
3,"NC_012920.1 Homo sapiens mitochondrion, comple...",785,808,24,-,-1.210,-1.16,1,25,CGCTTAGCCTAGCCACACCCCCAC,...,NaN,685,908,735,858,146,193,4,685,908
4,"NC_012920.1 Homo sapiens mitochondrion, comple...",954,984,31,-,-1.390,-1.48,1,25,CACCCCCTCCCCAATAAAGCTAAAACTCACC,...,NaN,854,1084,904,1034,683,146,5,854,1084
5,"NC_012920.1 Homo sapiens mitochondrion, comple...",1667,1707,41,-,-1.020,-1.48,1,25,CTGAGCTAAACCTAGCCCCAAACCCACTCCACCTTACTACC,...,more than in africa,1567,1807,1617,1757,350,683,6,1567,1807
6,"NC_012920.1 Homo sapiens mitochondrion, comple...",2057,2079,23,-,-1.520,-1.40,1,25,CCCACAGAACCCTCTAAATCCCC,...,NaN,1957,2179,2007,2129,133,350,7,1957,2179
7,"NC_012920.1 Homo sapiens mitochondrion, comple...",2212,2236,25,-,-1.000,-1.00,1,25,CAACACCCACTACCTAAAAAATCCC,...,NaN,2112,2336,2162,2286,912,133,8,2112,2336
10,"NC_012920.1 Homo sapiens mitochondrion, comple...",3471,3604,134,-,-1.170,-2.76,1,25,CTTCACCAAAGAGCCCCTAAAACCCGCCACATCTACCATCACCCTC...,...,5 overlapping g4 in africa,3371,3704,3421,3654,270,249,11,3371,3704
11,"NC_012920.1 Homo sapiens mitochondrion, comple...",3874,3909,36,-,-0.917,-1.28,1,25,CTAGCAGAGACCAACCGAACCCCCTTCGACCTTGCC,...,NaN,3774,4009,3824,3959,42,270,12,3774,3951
12,"NC_012920.1 Homo sapiens mitochondrion, comple...",3951,3984,34,-,-0.882,-1.20,1,25,CGCCGCAGGCCCCTTCGCCCTATTCTTCATAGCC,...,more than africa,3851,4084,3901,4034,102,42,13,3909,4084
15,"NC_012920.1 Homo sapiens mitochondrion, comple...",4228,4260,33,-,-1.210,-1.28,1,25,CCCATTACAATCTCCAGCATTCCCCCTCAAACC,...,NaN,4128,4360,4178,4310,112,67,16,4161,4360


In [58]:
# df_25bp['start'] = df_25bp['start'].apply(lambda x: x-25)
# df_25bp['end'] = df_25bp['end'].apply(lambda x: x+25)

In [10]:
data_check_start = []

for start, end, G4_id in zip(df_25bp['start'], 
                             df_25bp['end'], 
                             df_25bp['G4_id']):
    
    number_of_win = 0
    
    for i in range(start-25, end+25):
        row = []
        
        if i+WINDOW_SIZE <= end+25:
            window = [*range(i, i + WINDOW_SIZE)]       
        else:
            break

        window_seq = ''
        for pos in window:
            if pos != 3107:
                window_seq += str(ref_annot[ref_annot['Pos'] == pos].Nuc.values[0])
            else:
                window_seq += ''
        
        # print(i, window, WINDOW_SIZE, end)  
        
        number_of_win += 1

        row.append(G4_id)
        row.append('check_start')
        row.append(number_of_win)
        row.append(window[0])
        row.append(window[-1])
        row.append(window_seq)
        
        data_check_start.append(row)
# print(data_check_start)

In [15]:
data_check_end = []

for start, end, G4_id in zip(df_25bp['start'], 
                             df_25bp['end'], 
                             df_25bp['G4_id']):
    
    number_of_win = 0
    
    for i in range(end+25, start-25, -1):
        row = []

        
        if i+WINDOW_SIZE > end+25:
            continue  
        elif (i+WINDOW_SIZE <= end+25):
            window = [*range(i, i + WINDOW_SIZE)]       
        else:
            continue

        window_seq = ''
        for pos in window:
            if pos != 3107:
                window_seq += str(ref_annot[ref_annot['Pos'] == pos].Nuc.values[0])
            else:
                window_seq += ''
        
        number_of_win += 1

        row.append(G4_id)
        row.append('check_end')
        row.append(number_of_win)
        row.append(window[0])
        row.append(window[-1])
        row.append(window_seq)
        
        data_check_end.append(row)
# print(data_check_end)

In [16]:
data_check_sides = data_check_start + data_check_end

In [24]:
# data_check_sides

In [18]:
df_win_check_sides = pd.DataFrame(data=data_check_sides, 
                             columns = ['G4_id', 'side', 'number_of_win', 'window_start', 'window_end', 'sequence'])

In [23]:
df_win_check_sides[(df_win_check_sides['side'] == 'check_end') & (df_win_check_sides['G4_id'] == 3)]

,G4_id,side,number_of_win,window_start,window_end,sequence
4901,3,check_end,1,607,616,CACTGAAAAT
4902,3,check_end,2,606,615,ACACTGAAAA
4903,3,check_end,3,605,614,TACACTGAAA
4904,3,check_end,4,604,613,ATACACTGAA
4905,3,check_end,5,603,612,AATACACTGA
...,...,...,...,...,...,...
5110,3,check_end,210,398,407,TTTTATCTTT
5111,3,check_end,211,397,406,ATTTTATCTT
5112,3,check_end,212,396,405,AATTTTATCT
5113,3,check_end,213,395,404,AAATTTTATC


In [25]:
nuc_count_win_check_sides = df_win_check_sides['sequence'].apply(lambda x: Counter(x))

In [26]:
nuc_count_df_win_check_sides = pd.DataFrame.from_records(nuc_count_win_check_sides).sort_index(axis=1)

In [27]:
df_win_check_sides = df_win_check_sides.join(nuc_count_df_win_check_sides).fillna(0)

In [28]:
df_win_check_sides

,G4_id,side,number_of_win,window_start,window_end,sequence,A,C,G,T
0,3,check_start,1,393,402,TCAAATTTTA,4.0,1.0,0.0,5.0
1,3,check_start,2,394,403,CAAATTTTAT,4.0,1.0,0.0,5.0
2,3,check_start,3,395,404,AAATTTTATC,4.0,1.0,0.0,5.0
3,3,check_start,4,396,405,AATTTTATCT,3.0,1.0,0.0,6.0
4,3,check_start,5,397,406,ATTTTATCTT,2.0,1.0,0.0,7.0
...,...,...,...,...,...,...,...,...,...,...
9743,73,check_end,74,16324,16333,TTACCGTACA,3.0,3.0,1.0,3.0
9744,73,check_end,75,16323,16332,TTTACCGTAC,2.0,3.0,1.0,4.0
9745,73,check_end,76,16322,16331,ATTTACCGTA,3.0,2.0,1.0,4.0
9746,73,check_end,77,16321,16330,CATTTACCGT,2.0,3.0,1.0,4.0


In [30]:
# df_win_check_sides.to_csv('../data/processed/df_win_check_sides.csv', index=False)

Collect mutations from cancer dataset

In [32]:
df_cancers = pd.read_csv('../data/raw/mutations.csv')

In [33]:
df_win_check_sides = pd.read_csv('../data/processed/df_win_check_sides.csv')

In [39]:
Mut_win_check_sides = []
for window_start, window_end in zip(df_win_check_sides['window_start'], df_win_check_sides['window_end']):
    row = [*range(window_start, window_end)]
    Mut_win_check_sides.append(str(df_cancers[df_cancers['Pos'].isin(row)].Mut.values))

In [40]:
Mut_win_df_check_sides = pd.DataFrame(Mut_win_check_sides)

In [41]:
Mut_win_df_check_sides[0] = Mut_win_df_check_sides[0].str.findall("[A-T]>[A-T]")

In [42]:
obsMutwin_check_sides = pd.DataFrame.from_records(Mut_win_df_check_sides[0].apply(lambda x: Counter(x))) \
    .fillna(0) \
    .sort_index(axis=1)

In [43]:
df_win_cancer_check_sides = df_win_check_sides.join(obsMutwin_check_sides)

In [44]:
df_win_cancer_check_sides['number_of_Mut'] = df_win_cancer_check_sides[['A>C','A>G','A>T',
                                                              'C>A','C>G','C>T',
                                                              'G>A','G>C','G>T',
                                                              'T>A','T>C','T>G']].sum(axis=1)

In [45]:
df_win_cancer_check_sides.head()

,G4_id,side,number_of_win,window_start,window_end,sequence,A,C,G,T,...,C>A,C>G,C>T,G>A,G>C,G>T,T>A,T>C,T>G,number_of_Mut
0,3,check_start,1,393,402,TCAAATTTTA,4.0,1.0,0.0,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,3,check_start,2,394,403,CAAATTTTAT,4.0,1.0,0.0,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,check_start,3,395,404,AAATTTTATC,4.0,1.0,0.0,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
3,3,check_start,4,396,405,AATTTTATCT,3.0,1.0,0.0,6.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
4,3,check_start,5,397,406,ATTTTATCTT,2.0,1.0,0.0,7.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0


In [47]:
# df_win_cancer_check_sides.to_csv('../data/processed/df_win_cancer_check_sides.csv', index=False)